In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
%cd ../

/workspace


# Import Module

In [2]:
import json
import os
from collections import OrderedDict, defaultdict
from functools import partial

import numpy as np
import pandas as pd
import torch
from joblib import Parallel, delayed
from pyserini.search.lucene import LuceneSearcher
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import AutoConfig, AutoModel, AutoTokenizer

from src.datasets import Dataset, bert_collate_fn

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Load Data

In [3]:
train_data_path = "./data/train.json"
test_data_path = "./data/test_data.json"
with open(train_data_path, "r") as f1, open(test_data_path, "r") as f2:
    train_data = json.load(f1)
    test_data = json.load(f2)
test_question = pd.read_csv("./data/test_questions.csv", encoding="utf8")
sample = pd.read_csv("./data/sample_submission.csv", encoding="utf8")
print(f"# of train data: {len(train_data['data']):,}")
print(f"# of test data: {len(test_data['data']):,}")

# of train data: 1,862
# of test data: 827


In [4]:
train_queries = OrderedDict()
train_query_to_docs = defaultdict(list)
train_docs = OrderedDict()

for data in train_data["data"]:
    for paragraph in data["paragraphs"]:
        for q in paragraph["qas"]:
            train_queries[q["question_id"]] = q["question"]
            train_query_to_docs[q["question_id"]].append(paragraph["paragraph_id"])
        train_docs[paragraph["paragraph_id"]] = paragraph["context"]

print(f"# of queries: {len(train_queries):,}")
print(f"# of documents: {len(train_docs):,}")

# of queries: 233,121
# of documents: 137,335


In [5]:
query_lengths = np.array([len(q) for q in train_queries.values()])
print(f"Max length of query: {np.max(query_lengths):,}")
print(f"Min length of query: {np.min(query_lengths):,}")
print(f"Avg. length of query: {np.mean(query_lengths):.2f}")
print(f"Std. length of query: {np.std(query_lengths):.2f}")
print("-" * 40)
doc_lengths = np.array([len(d) for d in train_docs.values()])
print(f"Max length of document: {np.max(doc_lengths):,}")
print(f"Min length of document: {np.min(doc_lengths):,}")
print(f"Avg. length of document: {np.mean(doc_lengths):.2f}")
print(f"Std. length of document: {np.std(doc_lengths):.2f}")

Max length of query: 176
Min length of query: 6
Avg. length of query: 41.20
Std. length of query: 12.99
----------------------------------------
Max length of document: 676
Min length of document: 353
Avg. length of document: 508.54
Std. length of document: 72.12


---

# BM25 Baseline

In [33]:
def get_contents(data):
    contents = []
    for d in data["data"]:
        for paragraph in d["paragraphs"]:
            contents.append(
                {"id": paragraph["paragraph_id"], "contents": paragraph["context"]}
            )
    return contents


data_list = [
    (train_data, "./data/index/train/doc.json"),
    (test_data, "./data/index/test/doc.json"),
]

for data, index_path in data_list:
    contents = get_contents(data)
    os.makedirs(os.path.dirname(index_path), exist_ok=True)
    with open(index_path, "w", encoding="utf8") as f:
        json.dump(contents, f)

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input data/index/train/ \
  --language ko \
  --index data/train.index \
  --generator DefaultLuceneDocumentGenerator \
  --threads 16 \
  --storePositions --storeDocvectors --storeRaw

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input data/index/test/ \
  --language ko \
  --index data/test.index \
  --generator DefaultLuceneDocumentGenerator \
  --threads 16 \
  --storePositions --storeDocvectors --storeRaw

In [35]:
searcher = LuceneSearcher("./data/test.index")
searcher.set_language("ko")

In [54]:
answer = []
for q in tqdm(test_question["question_text"]):
    answer.append(",".join([r.docid for r in searcher.search(q, k=10)]))
sample["paragraph_id"] = answer
sample.to_csv("submission.csv", index=False)

  0%|          | 0/11434 [00:00<?, ?it/s]

---

# monoBERT

## First Stage Retrieval

In [23]:
queries = list(train_queries.values())
train_query_df = pd.DataFrame(data={"q_id": np.arange(len(queries)), "query": queries})

batch_size = 10000
num_batches = (len(train_query_df) + batch_size - 1) // batch_size

for n in range(num_batches):
    tsv_path = f"./data/top1000/train_queries_{n:02d}.tsv"
    os.makedirs(os.path.dirname(tsv_path), exist_ok=True)
    train_query_df[n : (n + 1) * batch_size].to_csv(
        tsv_path, index=False, header=None, sep="\t"
    )

In [ ]:
!python -m pyserini.search.lucene \
  --index data/train.index \
  --topics data/top1000/train_queries_00.tsv \
  --output data/top1000/train_top1000_00.txt \
  --bm25 --language ko --hits 1000 --batch-size 256 --threads 32

## PLM

In [67]:
pretrained_model_name = 'monologg/koelectra-base-v3-discriminator'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
bert = AutoModel.from_pretrained(pretrained_model_name)

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [186]:
topk_1000 = [r.docid for r in searcher.search(queries[0], k=1000)]

In [200]:
dataset = Dataset(
    train_queries, train_docs, train_query_to_docs, searcher, topk=1000, num_neg=1
)

In [201]:
dataset[0]

('공기역학적 방사능 중간직경은 무엇을 분석하여 얻었어',
 ['방사성핵종 별 공기 부피당 방사능 농도(Bq/m3 )는 부피당 분진 농도(μg/L)와 질량당 방사능 농도(Bq/kg) 값을 이용하여 계산 가능하다. 분진이 흩날릴 가능성이 있는 작업 지점에 설치한 광학입자계수기(OPC, Optical Particle Counter)를 활용한 계측과 채취시료의 방사능분석 의뢰 등을 통하여 이 값들을 각각 얻을 수 있다. 선량환산인자(DCF, Dose Conversion Factor)는 원료물질 등이 함유한 방사성핵종의 종류 외에 분진 입자의 크기 및 분포, 밀도, 기하학적 모양, 흡수형태 등 물리화학적 형태에 따라 그 값이 크게 달라진다. 입자의 크기와 분포는 앞서 언급한 OPC를 분석하여 얻은 공기역학적 방사능 중간직경(AMAD, Activity Median Aerodynamic Diameter)과 직경분포에 따른 분산(GSD, Geometric Standard Deviation)으로 결정되며, 입자의 밀도는 Pycnometer 등의 밀도측정장비로 얻을 수 있다. 모양인자는 1로 통일하여 적용되는데 이 값은 1에 가까운 구형일수록 보수적인 평가가 이루어진다는 해외 연구결과를 바탕으로 볼 때 합리적이다.',
  '셋째, 재난안전 R&D는 그 무엇보다도 학제적 접근이 필요하다. 재난은 단순히 과학기술의 문제가 아닌 모든 사회 문제가 결합된 융･복합적 사회 현상이다. 지진의 예측을 위해서는 지질학적 연구가 필요하지만, 지진을 대비하기 위해서는 내진 설계가 필요하고, 사회적으로 내진 설계를 요구하기 위해서는 효과적 규제 정책과 이를 감당할 수 있는 사회적 비용에 대한 철저한 분석이 필요하다. 또한 지진을 경험한 우리 국민들의 공포를 이해할 수 있는 심리적 접근과 함께 공동체가 재난 상황에서 효과적으로 협력하기 위한 사회학적 접근도 필수이다. 이처럼 재난이 융･복합적 성격을 가진 만큼 관련 R&D 역시 융･복합적일 필요가 있다. 현재 우리나라의 R&D는 철저히 과학기술적 

In [156]:
inputs = tokenizer(
    [train_queries[q_id] for q_id in q_ids[:32]],
    [train_docs[train_query_to_docs[q_id][0]] for q_id in q_ids[:32]],
    return_tensors="pt",
    padding='max_length',
    truncation='longest_first',
    max_length=512,
    # max_length=,
    # truncation='',
    # max_length=5,
    # truncation='only_first',
    # max_length=40,
    # padding=True,
    # truncation="only_first",
)

In [221]:
dataloader = DataLoader(
    dataset,
    batch_size=32,
    collate_fn=partial(bert_collate_fn, tokenizer=tokenizer, max_length=512),
    num_workers=32,
)

In [ ]:
for _ in tqdm(dataloader):
    pass

  0%|          | 0/7286 [00:02<?, ?it/s]